In [1]:
from New2048 import Game2048 as _2048
import numpy as np
import New2048

In [2]:
game = _2048(4)
game.start_game
game.playAdversaryAction()
#game.playRandUser()
#print(game)
#print(game.board_value())

+------+------+------+------+
|      |      |      |      |
+------+------+------+------+
|      |      |      |      |
+------+------+------+------+
|  2   |      |      |      |
+------+------+------+------+
|      |      |      |      |
+------+------+------+------+
4


In [3]:
#New2048.main()

# TEST 1 - Monte Carlo Tree Search

the below code is our initial code for the monte carlo tree search, the test done in this notebook is only 4x4 grids while the others are for 5x5 and 6x6 notebooks respectivly



In [4]:
# Starter Code: MCTS Implementation
import time


class Tree():
    def __init__(self, *, start_grid=None, parent=None, move=None):
        if parent is None:
            self.parent = None
            self.move = None
            self.grid = start_grid
        else:
            self.parent = parent
            self.move = move
            self.grid = parent.grid.copy()
            if self.grid.play_state:
                self.grid.playUserAction(move)
            else:
                self.grid.playAdversaryAction()
        
        self.values = []
        self.n = 0
        if self.is_terminal_state:
            self.unexplored_moves = set()
        else:
            if self.grid.play_state:
                self.unexplored_moves = set(self.grid.get_user_actions())
            else:
                #print(self.grid.getAdverseryActions())
                self.unexplored_moves = set(self.grid.getAdverseryActions())
        self.children = set([])
            
    @property
    def fully_expanded(self):
        return len(self.unexplored_moves) == 0
    
    @property
    def is_terminal_state(self):
        return (self.grid.winner is not None)


def monte_carlo_tree_search(start_grid, num_iterations=1000):
    """MCTS core loop"""
    # Start by creating the root of the tree.
    root = Tree(start_grid=start_grid)
    
    # Loop through MCTS iterations.
    for _ in range(num_iterations):
        # One step of MCTS iteration
        leaf = traverse(root)
        sim_res = rollout(leaf, start_grid)
        backpropagate(leaf, sim_res)

    # When done iterating, return the 'best' child of the root node.
    return best_child(root).move

def best_child(node):
    """When done sampling, pick the child visited the most."""
    #print(node)
    return max(node.children, key=lambda child: child.n)

'''def worst_child(node):
    """ Exact opposite of the best child (Least visited node) """
    return min(node.children, key=lambda child: child.n)'''


def best_uct(node, C=5):
    """Pick the best action according to the UCB/UCT algorithm"""
    chos_val = float("-inf")
    for nub in node.children:
        temp = np.sum(nub.values)/nub.n + C * np.sqrt(np.log(node.n)/nub.n)
        if temp >chos_val:
            chos_val = temp
            chose = nub
    try:
        return chose
    except:
        print(node.grid)
        return chose


def traverse(node):
    # If fully explored, pick one of the children
    while node.fully_expanded and not node.is_terminal_state:
        node = best_uct(node)

    # If the node is terminal, return it
    if node.is_terminal_state:
        return node
    
    # If the node is not terminal:
    # 1. pick a new move from 'unexplored_moves'
    move = node.unexplored_moves.pop()
    # 2. create a new child
    new_child = Tree(parent=node, move=move)
    # 3. add that child to the list of children
    node.children.add(new_child)
    # 4. return that new child
    return new_child


def rollout(node, start_grid):
    sample = node.grid.copy()
    winner = sample.play_random_moves_until_done()
    '''if winner == 0:
        return 0
    elif winner == start_grid.play_state:
        return 1
    else:
        return -1'''
    return sample.board_value()/(2048 ** 2)


def backpropagate(node, simulation_result):
    """Update the node and its parent (via recursion)."""
    if node is None:
        return
    node.n += 1
    node.values.append(simulation_result)
    backpropagate(node.parent, simulation_result)
    

In [18]:

time_arr = []
win_arr = []
wins = 0
for i in range(100):
    board = _2048(4)
    tot_time = 0
    while board.winner == None:
        #print(board)
        if board.play_state:
            stime = time.time()
            action = monte_carlo_tree_search(board, 250)
            tot_time += stime
            board.playUserAction(action)
            board.play_state = False
        else:
            board.playAdversaryAction()
            board.play_state = True
    if board.winner == 1:
        wins += 1
        win_arr.append(1)
    else:
        win_arr.append(0)
    time_arr.append(tot_time)
    print(i)
    print(board)
    print(board.winner)
    print(board.board_value())
    print(tot_time)
print("wins - " + str(wins))
print("percent - " +str(wins/100))
print(time_arr)
    

0
+------+------+------+------+
|  2   |  32  |  2   |  4   |
+------+------+------+------+
|  16  | 1024 |  16  | 128  |
+------+------+------+------+
|  4   |  32  | 512  |  4   |
+------+------+------+------+
|  2   |  4   |  8   |  2   |
+------+------+------+------+
2
1329808.0
153214710464.22263
1
+------+------+------+------+
|  2   | 2048 |      | 1024 |
+------+------+------+------+
|  4   |  8   |  4   |      |
+------+------+------+------+
|  16  |  32  | 128  |  8   |
+------+------+------+------+
|  2   |  4   |  16  |  4   |
+------+------+------+------+
1
5261000.0
131083697658.1701
2
+------+------+------+------+
|  16  |  2   |  8   |  64  |
+------+------+------+------+
|  2   |  16  |  4   |  2   |
+------+------+------+------+
|  8   | 256  |  16  |  64  |
+------+------+------+------+
|  2   |  8   |  64  |  4   |
+------+------+------+------+
2
78832.0
197476740968.37006
3
+------+------+------+------+
|  8   |  4   |  2   |  4   |
+------+------+------+------+
| 

28
+------+------+------+------+
|  2   |  32  |  16  |  2   |
+------+------+------+------+
|  4   | 128  |  4   |  64  |
+------+------+------+------+
|  8   |  32  |  8   |  4   |
+------+------+------+------+
|  2   |  64  |  4   |  2   |
+------+------+------+------+
2
27088.0
163429064738.66992
29
+------+------+------+------+
|  8   |  4   |  2   |  8   |
+------+------+------+------+
|  2   | 128  |  4   |  2   |
+------+------+------+------+
|  4   |  32  | 256  |  16  |
+------+------+------+------+
|  2   |  4   |  8   |  2   |
+------+------+------+------+
2
83476.0
120869413342.80177
30
+------+------+------+------+
|  4   |  2   | 128  |  2   |
+------+------+------+------+
|  2   |  32  |  16  |  4   |
+------+------+------+------+
|  32  | 256  |  4   |  8   |
+------+------+------+------+
|  2   |  4   |  8   |  4   |
+------+------+------+------+
2
84448.0
170238611469.36172
31
+------+------+------+------+
|  64  |  2   |  32  |  2   |
+------+------+------+------+
|

56
+------+------+------+------+
|  4   |  8   |  64  |  4   |
+------+------+------+------+
|  32  | 128  |  16  |  8   |
+------+------+------+------+
|  4   |  16  |  8   |  64  |
+------+------+------+------+
|  2   |  8   |  2   |  8   |
+------+------+------+------+
2
26488.0
78309779746.80614
57
+------+------+------+------+
|  2   |  16  |  2   |  16  |
+------+------+------+------+
|  8   |  4   |  32  |  4   |
+------+------+------+------+
|  2   | 128  |  16  |  8   |
+------+------+------+------+
|  16  |  2   |  8   |  2   |
+------+------+------+------+
2
18676.0
59583528194.54465
58
+------+------+------+------+
|  8   |  4   |  2   |  4   |
+------+------+------+------+
|  4   | 512  |  16  |  8   |
+------+------+------+------+
|  8   |  16  |  64  |  4   |
+------+------+------+------+
|  2   |  8   | 128  |  2   |
+------+------+------+------+
2
283468.0
214500702790.9774
59
+------+------+------+------+
|  32  |  4   |  8   |  4   |
+------+------+------+------+
|  

84
+------+------+------+------+
|  4   |  16  |  4   |  2   |
+------+------+------+------+
| 128  |  4   |  16  |  4   |
+------+------+------+------+
|  64  |  2   | 128  |  16  |
+------+------+------+------+
|  2   |  32  |  4   |  2   |
+------+------+------+------+
2
38752.0
68095478081.47228
85
+------+------+------+------+
|  4   |  2   |  32  |  2   |
+------+------+------+------+
|  16  | 128  |  64  |  8   |
+------+------+------+------+
|  4   | 256  |  8   |  16  |
+------+------+------+------+
|  2   |  16  |  4   |  2   |
+------+------+------+------+
2
88000.0
143000504542.0773
86
+------+------+------+------+
|  4   |  2   |  16  |  2   |
+------+------+------+------+
|  16  |  8   | 128  |  8   |
+------+------+------+------+
|  2   |  64  |  8   |  4   |
+------+------+------+------+
|  4   |  32  |  4   |  2   |
+------+------+------+------+
2
22288.0
49369222037.75933
87
+------+------+------+------+
|  2   |  4   |  32  |  4   |
+------+------+------+------+
|  3

In [19]:
f = open("res.csv", 'w')

for i in range(len(time_arr)):
    f.write(str(time_arr[i]) + ", " + str(win_arr[i]) + ",\n")
f.close()

# TEST 2 - n-Step MCTS

This Test was to modify the MCTS algorthm with n-step bootstraping durring the rollout phase. From our results we where able to get a significant speen boost however this was at the cost of accuracy. we concluded that limiting the rewards that can be recieved seen by MCTS results in a preformence decrease

In [5]:

def mcts_with_n_step(start_grid, n_step, num_iterations = 1000):
    
    # Start by creating the root of the tree.
    root = Tree(start_grid=start_grid)
    
    # Loop through MCTS iterations.
    for _ in range(num_iterations):
        # One step of MCTS iteration
        leaf = traverse(root)
        sim_res = rollout_n_step(leaf, start_grid, n_step)
        backpropagate(leaf, sim_res)

    # When done iterating, return the 'best' child of the root node.
    return best_child(root).move


def rollout_n_step(node, start_grid, n_step):
    sample = node.grid.copy()
    winner = sample.play_random_moves_for_n(n_step)

    return sample.board_value()/(2048 ** 2)




In [7]:
 time_arr = []
win_arr = []
wins = 0
for i in range(100):
    board = _2048(4)
    tot_time = 0
    while board.winner == None:
        #print(board)
        if board.play_state:
            stime = time.time()
            action = mcts_with_n_step(board, 100, 250)
            tot_time += stime
            board.playUserAction(action)
            board.play_state = False
        else:
            board.playAdversaryAction()
            board.play_state = True
    if board.winner == 1:
        wins += 1
        win_arr.append(1)
    else:
        win_arr.append(0)
    time_arr.append(tot_time)
    print(i)
    print(board)
    print(board.winner)
    print(board.board_value())
    print(tot_time)
print("wins - " + str(wins))
print("percent - " +str(wins/100))
print(time_arr)

0
+------+------+------+------+
|  16  |  8   |  64  |  2   |
+------+------+------+------+
|  2   |  32  | 512  | 256  |
+------+------+------+------+
|  16  | 128  |  32  |  8   |
+------+------+------+------+
|  4   |  2   |  4   |  2   |
+------+------+------+------+
2
350896.0
209399003412.29755
1
+------+------+------+------+
|  2   |  16  |  8   |  4   |
+------+------+------+------+
|  8   |  32  |  4   |  2   |
+------+------+------+------+
|  32  |  8   |  16  |  64  |
+------+------+------+------+
|  8   | 256  |  4   |  2   |
+------+------+------+------+
2
72508.0
154921215497.19693
2
+------+------+------+------+
|  4   |  16  |  4   |  2   |
+------+------+------+------+
|  8   | 256  |  8   |  16  |
+------+------+------+------+
|  2   |  32  | 128  |  2   |
+------+------+------+------+
|  4   |  64  |  8   |  4   |
+------+------+------+------+
2
87820.0
156623647610.9081
3
+------+------+------+------+
|  2   |  4   |  32  |  2   |
+------+------+------+------+
|  64

28
+------+------+------+------+
|  4   |  2   |  4   |  16  |
+------+------+------+------+
|  8   |  4   |  8   |  2   |
+------+------+------+------+
|  4   |  32  | 256  |  16  |
+------+------+------+------+
|  2   |  16  |  4   |  2   |
+------+------+------+------+
2
67552.0
100443450393.41965
29
+------+------+------+------+
|  2   |  16  |  2   |  8   |
+------+------+------+------+
| 128  | 512  |  4   |  2   |
+------+------+------+------+
|  2   |  4   | 256  |  16  |
+------+------+------+------+
| 128  |  16  |  64  |  4   |
+------+------+------+------+
2
365440.0
153218823469.83817
30
+------+------+------+------+
|  64  |  2   |  16  |  2   |
+------+------+------+------+
|  2   | 128  | 256  |  16  |
+------+------+------+------+
|  16  |  8   |  4   |  32  |
+------+------+------+------+
|  2   |  64  |  2   |  4   |
+------+------+------+------+
2
92020.0
183862589743.99823
31
+------+------+------+------+
|  2   |  4   |  8   |  4   |
+------+------+------+------+


56
+------+------+------+------+
|  2   |  4   |  64  |  2   |
+------+------+------+------+
|  16  | 256  |  32  |  8   |
+------+------+------+------+
|  32  | 1024 | 512  |  4   |
+------+------+------+------+
|  2   |  4   |  16  |  2   |
+------+------+------+------+
2
1383040.0
229828285801.17957
57
+------+------+------+------+
|  2   |  4   |  16  |  2   |
+------+------+------+------+
|  8   |  32  | 256  |  8   |
+------+------+------+------+
|  4   |  8   | 512  |  4   |
+------+------+------+------+
|  2   |  4   |  64  |  2   |
+------+------+------+------+
2
333328.0
214506402979.79758
58
+------+------+------+------+
|  4   |  2   | 128  |  2   |
+------+------+------+------+
| 512  |  64  |  32  |  8   |
+------+------+------+------+
|  2   |  8   |  16  | 512  |
+------+------+------+------+
| 512  |  2   | 1024 |  4   |
+------+------+------+------+
2
1856944.0
265579360150.40973
59
+------+------+------+------+
|  4   |  8   |  2   | 128  |
+------+------+------+----

84
+------+------+------+------+
|  4   |  8   |  4   |  2   |
+------+------+------+------+
|  2   | 256  |  32  |  4   |
+------+------+------+------+
|  8   |  2   |  64  | 128  |
+------+------+------+------+
|  2   |  32  |  16  |  2   |
+------+------+------+------+
2
88516.0
173648084193.1414
85
+------+------+------+------+
|  8   |  4   |  8   |  2   |
+------+------+------+------+
|  4   | 256  |  16  |  4   |
+------+------+------+------+
|  2   |  4   | 512  | 128  |
+------+------+------+------+
|  16  |  2   |  16  |  4   |
+------+------+------+------+
2
345052.0
299628071447.67413
86
+------+------+------+------+
| 512  |  8   |  2   |  32  |
+------+------+------+------+
|  2   | 256  |  64  |  4   |
+------+------+------+------+
|  4   | 2048 |  16  |  2   |
+------+------+------+------+
|      |      | 512  |      |
+------+------+------+------+
1
4789612.0
195779709790.08484
87
+------+------+------+------+
|  2   |  8   |  4   |  2   |
+------+------+------+------+

In [8]:
f = open("res_boot.csv", 'w')

for i in range(len(time_arr)):
    f.write(str(time_arr[i]) + ", " + str(win_arr[i]) + ",\n")
f.close()

# TEST 3 - Rand Walk

the below is a constant case where we check how well the MCTS runs in comparison to this

In [10]:
time_arr = []
win_arr = []
wins = 0
for i in range(100):
    board = _2048(4)
    tot_time = 0
    board.playAdversaryAction()
    board.play_state = True
    while board.check_game_over() == False:
        #print(board)
        if board.play_state:
            board.playRandUser()
            board.play_state = False
        else:
            board.playAdversaryAction()
            board.play_state = True

    win_arr.append(board.get_highest_tile())
    time_arr.append(tot_time)
    print(board.get_highest_tile())
    print(board)
    print(board.winner)
    print(board.board_value())
    print(tot_time)
print("wins - " + str(wins))
print("percent - " +str(wins/100))
print(time_arr)

128.0
+------+------+------+------+
|  2   |  8   |  4   |  2   |
+------+------+------+------+
|  16  |  4   |  16  |  64  |
+------+------+------+------+
|  4   |  64  | 128  |  8   |
+------+------+------+------+
|  2   |  16  |  8   |  2   |
+------+------+------+------+
2
25600.0
0
128.0
+------+------+------+------+
|  4   |  2   |  4   |  2   |
+------+------+------+------+
|  64  |  16  |  32  |  8   |
+------+------+------+------+
|  8   | 128  |  16  |  2   |
+------+------+------+------+
|  2   |  4   |  2   |  8   |
+------+------+------+------+
2
22276.0
0
256.0
+------+------+------+------+
|  2   |  8   |  16  |  4   |
+------+------+------+------+
|  4   |  32  | 256  |  16  |
+------+------+------+------+
|  64  |  8   |  32  |  4   |
+------+------+------+------+
|  4   |  2   |  4   |  2   |
+------+------+------+------+
2
72412.0
0
128.0
+------+------+------+------+
|  2   | 128  |  2   |  4   |
+------+------+------+------+
|  4   |  2   |  64  |  16  |
+------+--

128.0
+------+------+------+------+
|  8   |  4   |  2   |  4   |
+------+------+------+------+
|  4   |  32  |  64  |  8   |
+------+------+------+------+
|  16  | 128  |  16  |  4   |
+------+------+------+------+
|  2   |  4   |  32  |  2   |
+------+------+------+------+
2
23260.0
0
128.0
+------+------+------+------+
|  16  |  32  |  4   |  2   |
+------+------+------+------+
|  8   | 128  |  64  |  32  |
+------+------+------+------+
|  4   |  16  |  4   |  16  |
+------+------+------+------+
|  2   |  4   |  8   |  2   |
+------+------+------+------+
2
23500.0
0
128.0
+------+------+------+------+
|  4   |  64  |  4   |  2   |
+------+------+------+------+
|  2   |  16  | 128  |  32  |
+------+------+------+------+
|  16  |  32  |  8   |  2   |
+------+------+------+------+
|  2   |  4   |  2   |  16  |
+------+------+------+------+
2
23428.0
0
64.0
+------+------+------+------+
|  8   |  4   |  2   |  4   |
+------+------+------+------+
|  4   |  32  |  64  |  16  |
+------+---

64.0
+------+------+------+------+
|  2   |  8   |  2   |  8   |
+------+------+------+------+
|  8   |  2   |  8   |  64  |
+------+------+------+------+
|  4   |  8   |  64  |  8   |
+------+------+------+------+
|  2   |  4   |  32  |  2   |
+------+------+------+------+
2
9652.0
0
64.0
+------+------+------+------+
|  2   |  8   |  4   |  2   |
+------+------+------+------+
|  8   |  16  |  8   |  16  |
+------+------+------+------+
|  2   |  64  |  32  |  4   |
+------+------+------+------+
|  16  |  2   |  4   |  2   |
+------+------+------+------+
2
6148.0
0
64.0
+------+------+------+------+
|  2   |  4   |  16  |  4   |
+------+------+------+------+
|  4   |  8   |  4   |  16  |
+------+------+------+------+
|  16  |  64  |  2   |  4   |
+------+------+------+------+
|  4   |  16  |  4   |  2   |
+------+------+------+------+
2
5308.0
0
32.0
+------+------+------+------+
|  4   |  2   |  4   |  2   |
+------+------+------+------+
|  2   |  8   |  2   |  8   |
+------+------+--

In [11]:
f = open("res_rand4-score.csv", 'w')

for i in range(len(time_arr)):
    f.write(str(time_arr[i]) + ", " + str(win_arr[i]) + ",\n")
f.close()

# TEST 4 - How High

the below is a the MCTS but we allowed the game to be played past the goal 2048 and game boards where scored based on a sum of squares. from this we see that the MCTS algorthm is by far more successful however we see that the time taken to acheive these results are significant larger especially for the larger board sizes which we have concluded are an easier problem then 4x4 grids. 

We note that this method causes the accuracy of the MCTS on a 4x4 grid to have a signifant increase. our inital results show a 20% win rate on 100 games but with this modification we see 70% win rate on 10 games.

In [7]:
class Tree():
    def __init__(self, *, start_grid=None, parent=None, move=None):
        if parent is None:
            self.parent = None
            self.move = None
            self.grid = start_grid
        else:
            self.parent = parent
            self.move = move
            self.grid = parent.grid.copy()
            if self.grid.play_state:
                self.grid.playUserAction(move)
            else:
                self.grid.playAdversaryAction()
        
        self.values = []
        self.n = 0
        if self.is_terminal_state:
            self.unexplored_moves = set()
        else:
            if self.grid.play_state:
                self.unexplored_moves = set(self.grid.get_user_actions())
            else:
                #print(self.grid.getAdverseryActions())
                self.unexplored_moves = set(self.grid.getAdverseryActions())
        self.children = set([])
            
    @property
    def fully_expanded(self):
        return len(self.unexplored_moves) == 0
    
    @property
    def is_terminal_state(self):
        return (self.grid.check_game_over())
    
def monte_carlo_tree_search(start_grid, num_iterations=1000):
    """MCTS core loop"""
    # Start by creating the root of the tree.
    root = Tree(start_grid=start_grid)
    
    # Loop through MCTS iterations.
    for _ in range(num_iterations):
        # One step of MCTS iteration
        leaf = traverse(root)
        sim_res = rollout_o(leaf, start_grid)
        backpropagate(leaf, sim_res)

    # When done iterating, return the 'best' child of the root node.
    return best_child(root).move



def rollout_o(node, start_grid):
    sample = node.grid.copy()
    winner = sample.play_til_over()
    '''if winner == 0:
        return 0
    elif winner == start_grid.play_state:
        return 1
    else:
        return -1'''
    return sample.board_value()/(16384 ** 2)




In [8]:
time_arr = []
win_arr = []
wins = 0
for i in range(100):
    board = _2048(4)
    tot_time = 0
    board.playAdversaryAction()
    board.play_state = True
    while board.check_game_over() == False:
        #print(board)
        if board.play_state:
            stime = time.time()
            action = monte_carlo_tree_search(board, 250)
            tot_time += stime
            board.playUserAction(action)
            board.play_state = False
        else:
            board.playAdversaryAction()
            board.play_state = True

    win_arr.append(board.get_highest_tile())
    time_arr.append(tot_time)
    print(board.get_highest_tile())
    print(board)
    print(board.winner)
    print(board.board_value())
    print(tot_time)
print("wins - " + str(wins))
print("percent - " +str(wins/100))
print(time_arr)

4096.0
+------+------+------+------+
|  2   |  32  |  2   | 2048 |
+------+------+------+------+
|  8   | 128  | 4096 |  8   |
+------+------+------+------+
|  4   |  16  |  32  |  4   |
+------+------+------+------+
|  2   |  4   |  16  |  2   |
+------+------+------+------+
2
20990656.0
166839036491.18185
32768.0
+------+------+------+------+
|  4   |16384 |32768 |  2   |
+------+------+------+------+
|  8   |  4   |  8   |  16  |
+------+------+------+------+
|  4   |  64  | 128  |  32  |
+------+------+------+------+
|  2   |  32  |  8   |  4   |
+------+------+------+------+
2
1342200328.0
205995177159.20276
1024.0
+------+------+------+------+
|  8   |  16  |  2   |  4   |
+------+------+------+------+
|  2   |  64  |  4   |  2   |
+------+------+------+------+
|  8   |  32  | 1024 |  4   |
+------+------+------+------+
|  2   |  4   |  8   |  32  |
+------+------+------+------+
2
1055248.0
216209861186.1488
256.0
+------+------+------+------+
|  2   |  4   |  8   |  2   |
+-----

KeyboardInterrupt: 